In [39]:
import os
import sys

sys.path.append("../src")  # for src imports
import logging

import geopandas as gpd
from owslib.wms import WebMapService

from wms.get_wms_crops import get_crop_bboxes

In [ ]:
source_geometries_file = r"..\data\DE_wildlife_crossing.geojson"
output_folder = r"..\data\de_wms_export_v2"
wms_url = "https://image.discomap.eea.europa.eu/arcgis/services/GioLand/VHR_2021_LAEA/ImageServer/WMSServer/?request=GetCapabilities&service=WMS"
pixel_size = 2  # meter per pixel
buffer_size = 100  # meter

In [19]:
gdf = gpd.read_file(source_geometries_file)
gdf = gdf[
    (gdf.geom_type == "Polygon") | (gdf.geom_type == "Point") & (gdf["bridge"] == "yes")
]
print(f"Number of geometries to process: {len(gdf)}")

Number of geometries to download: 443


In [43]:
wms = WebMapService(
    wms_url,
    version="1.3.0",
)

In [6]:
layer_name = list(wms.contents)[0]
print(layer_name)

VHR_2021_LAEA


In [22]:
gdf_centroids = gdf.to_crs(epsg=3035).centroid
areas_of_interest = get_crop_bboxes(gdf_centroids, buffer_size)

443

In [ ]:
os.makedirs(output_folder, exist_ok=True)

In [47]:
# for every point create a 200x200m polygon around it and request a WMS crop centered on the point
img_format = "image/png"
img_size = (
    int(buffer_size * 2 / pixel_size),
    int(buffer_size * 2 / pixel_size),
)  # width, height in pixels

for idx, bbox in enumerate(areas_of_interest):
    # Request WMS image
    img = wms.getmap(
        layers=[layer_name],
        srs="EPSG:3035",
        bbox=bbox,
        size=img_size,
        format=img_format,
        transparent=True,
    )

    # Save image to file
    img_data = img.read()
    img_filename = os.path.join(output_folder, f"crossing_{idx}.png")
    with open(img_filename, "wb") as f:
        f.write(img_data)

    print(f"Saved WMS crop for crossing {idx} to {img_filename}")